In [ ]:
!pip install llama-index
!pip install -U sentence-transformers
!pip install --upgrade pip setuptools wheel
!pip install -U bitsandbytes
!pip install flash-attn --no-build-isolation
pip install bert-score
!pip install FlagEmbedding
!pip install accelerate -U
!pip install --upgrade huggingface-hub==0.24.0
!pip install transformers==4.46.3
!pip install bm25s[full]
!pip install PyStemmer
!pip install jax[cpu]

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = "t-tech/T-pro-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2"
).eval()

generation_config = GenerationConfig.from_pretrained(model_name)

2024-12-14 00:03:55.112816: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-14 00:03:55.112882: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-14 00:03:55.113971: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-14 00:03:55.119756: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-14 00:03:55.990599: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [3]:
chunk_size = 1300
chunk_overlap = 100

In [4]:
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser
import textwrap
from sentence_transformers import SentenceTransformer
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig
import torch
from FlagEmbedding import FlagReranker
import pandas as pd
import json
import numpy as np

splitter_classic = SentenceSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)


def split_sentences(text):
    sentence_endings = r'(?<=[.!?]) +'
    sentences = re.split(sentence_endings, text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

def split_into_chunks(doc, splitter):
    return splitter.split_text(doc)

def get_splitted(documents, splitter):
    output_with_indices = []
    for index, doc in enumerate(documents):
        chunks = split_into_chunks(doc, splitter)
        for chunk in chunks:
            output_with_indices.append((index, chunk))
    return output_with_indices

def rate(indexes, texts, ind, true_label):
    if indexes[ind] == true_label:
        return 1
    return 0

def rate_retriev(train, pipeline, metric=None):
    trues = 0
    for index, row in train.iterrows():
        if index % 50 == 0:
            print(index)
            
        q, y = row['question'], np.where(row['text'] == corpus)[0]
        txts = new_pipe(indexes, texts, q, k1=50, k2=5, p1=embedder_p, p2=get_top_k_chuks_rerank)
        trues += any([rate(indexes, texts, j, y) for j in txts])
        if not(any([rate(indexes, texts, j, y) for j in txts])):
            logs.append(
                [
                    index,
                    y,
                    txts
                ]
            )
    return trues / len(train)

def get_top_k_chuks_rerank(question, chunks_e, k=100, batch_size=10):
    scores = torch.tensor(reranker.compute_score([[question, chnk] for chnk in chunks_e], normalize=True) )
    reranged = reversed(scores.argsort())[:k]
    return reranged, scores[reranged] #индексы доков по убыванию релев, скоры
def embedder_p(q, chunks, k):
    res = retrieve_context_topk(embedder_model, chunks, q, k)
    return res, []
def retrieve_context_topk(embedder, texts, question, k=1, get_indices=False):
    question_embedding = embedder([question])[0]
    scores = np.dot(embeddings, question_embedding) / \
        (np.linalg.norm(embeddings, axis=1) * np.linalg.norm(question_embedding))
    top_k_indices = np.argsort(scores)[-k:]
    if get_indices:
        return [i for i in reversed(top_k_indices)]
    return [i for i in reversed(top_k_indices)]

def get_embedder(model_name):
    embedder = SentenceTransformer(model_name)

    def f(texts, **args):
        # Ensure texts is a list
        if isinstance(texts, str):
            texts = [texts]  # If a single string is provided, convert to list
        return embedder.encode(texts, normalize_embeddings=True, **args)
    
    return f

import re
def parse_questions(output):
    output = output.lower()
    question_pattern = r'вопрос \d+ *:.+'
    questions = re.findall(question_pattern, output)
    questions = [q[q.find(':')+1:].strip() for q in questions]
    return questions


def create_queries(model, tokenizer, generation_config, question):
    prompt_desired = f'''На основе вопроса: {question} создай 2-3 абсолютно вопроса схожих по смыслу, которые могли бы быть заданы в том же контексте но переписаны другими словами\n
                         Формат: Вопрос 1: ...\n Вопрос 2: ...\n Вопрос 3: ...'''
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": ""
    }, {
        "role": "user",
        "content": prompt_desired
    }], tokenize=False, add_generation_prompt=True)
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    try:
        questions_cands = parse_questions(output)
        questions_cands[-1] = question  
    except (Exception) as e:
        print(e)
        return [question]
    # print('Вопросы сгенерированы')
    return questions_cands

def reciprocal_rank_fusion(ranked_lists, k=60):
    scores = {}
    for rank_list in ranked_lists:
        for rank, item in enumerate(rank_list):
            if item not in scores:
                scores[item] = 0
            scores[item] += 1 / (rank + 1 + k)
    # print('Ранжирование пройдено')
    # print(sorted(scores.items(), key=lambda x: x[1], reverse=True))
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

def check_real(arr):
    return [indexes[el] for el in arr]

# def new_pipe(inds, texts, q, k1, k2, p1, p2):
#     qs = create_queries(
#         model,
#         tokenizer, 
#         generation_config,
#         q
#     )
#     list_of_ranks = []
#     for cand_q in qs:
#         indexes_f, _ = p1(cand_q, texts, k1)
#         template = texts[indexes_f]
#         ind2, _ = p2(cand_q, template, k2)
#         relev_chunks = template[ind2]
#         final_inds = [np.where(texts==rr)[0][0] for rr in relev_chunks]
#         list_of_ranks.append(final_inds)
#     #потому что (ind, score)
#     result_inds = [f.item() for f, _ in reciprocal_rank_fusion(list_of_ranks)][:k2]
#     return result_inds

def new_pipe(inds, texts, q, k1, k2, p1, p2, scores=False):
    # qs = create_queries(
    #     model,
    #     tokenizer, 
    #     generation_config,
    #     q
    # )
    
    qs = [q]
    list_of_ranks = []
    for cand_q in qs:
        indexes_f, _ = p1(cand_q, texts, k1)
        indexes_bm25, _ = get_top_k(cand_q, 1321321, k1)
        
        indexes_f = [f.item() for f, _ in reciprocal_rank_fusion(
            [indexes_f, indexes_bm25]
        )][:k1]
        
        template = texts[indexes_f]
        ind2, scorings_reranker = p2(cand_q, template, k2)
        relev_chunks = template[ind2]
        final_inds = [np.where(texts==rr)[0][0] for rr in relev_chunks]
        list_of_ranks.append(final_inds)
    #потому что (ind, score)
    if scores:
        return list_of_ranks[0], scorings_reranker
    return list_of_ranks[0], []


with open('webhelp_dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

if 'data' in data:
    df = pd.DataFrame(data['data'])

df = df[df.description.str.len() > 4]

df['text'] = df['title'] + ' ' + df['description']

texts_urls = df[['text', 'url']].drop_duplicates(subset='text', keep='first').reset_index(drop=True)
corpus = texts_urls.text.values
urls = texts_urls.url.values


In [8]:
import Stemmer
import bm25s
corpus = texts_urls.text.values
urls = texts_urls.url.values

corpus1 = get_splitted(corpus, splitter_classic)
indexes, texts = np.array([f for f, _ in corpus1]), np.array([s for _, s in corpus1])

stemmer = Stemmer.Stemmer("russian")


corpus_tokens = bm25s.tokenize(texts, stemmer=stemmer, stopwords="ru")
retriever = bm25s.BM25(method="bm25+", delta=1.5)
retriever.index(corpus_tokens)


def get_top_k(question, chunks, k) -> tuple[str, str]:
    questions_tokens = bm25s.tokenize(question, stemmer=stemmer, stopwords="ru")
    result_indexes, scores = retriever.retrieve(questions_tokens, k=k)

    result_indexes = result_indexes[0]

    return result_indexes, []


In [ ]:
#КОНТЕКСТУАЛЬНЫЕ ЧАНКИ, воркайте??
with open('logical_chunks_new_by_vikhr.json', 'r') as fl:
    chunks_logical = json.loads(fl.read())
    for ck_id, context_new in chunks_logical:
        texts[ck_id] = 'Контекст:' + context_new.lower().replace('контекст:', '') + '\n' + texts[ck_id]
        

In [10]:
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) 
embedder_model = get_embedder('deepvk/USER-bge-m3')
embeddings = embedder_model(texts, batch_size=32)

In [11]:
def call_pipe(indexes, texts, q, k1=50, k2=5, scores=False):
    ranked, scorings = new_pipe(indexes, texts, q, k1=k1, k2=k2, p1=embedder_p, p2=get_top_k_chuks_rerank, scores=scores)
    ranked_texts = [corpus[ind] for ind in check_real(ranked)]
    if scores:
        return ranked_texts, [ind for ind in check_real(ranked)], scorings 
    return ranked_texts, [ind for ind in check_real(ranked)]

In [12]:
def generate(model, tokenizer, generation_config, prompt_desired):
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": ""
    }, {
        "role": "user",
        "content": prompt_desired
    }], tokenize=False, add_generation_prompt=True)
    print('here')
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config, max_new_tokens=1524)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    return output

In [21]:
RAG_PROMPT = (
    "Контекстная информация снизу. (набор документов чанков)\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Используя только данную информацию контекст, "
    "Ответь на вопрос. Дай прямой ответ. не генерируй markdown. Если этих документов недостаточно, чтобы ответить на вопрос,отказывайся отвечать. Если вопрос не по теме домена сразу отказывайся отвечать.\n"
    "Вопрос: {query_str}\n"
    "Ответ: "
)

REASONING_PROMPT = (
        "Контекстная информация снизу. (набор документов чанков)\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Используя только данную информацию контекст и вопрос пользователя, "
        "Порассуждай сперва какие из этих чанков точно содержут ответ. В последнем предложении выдели (номеры) чанков которые точно содержат ответ. "
        "чанки идут по релевантности по убыванию."
        "Если вопрос подразумевает использование нескольких чанков, также выдели их. В конце дай номер релевантных чанков. Ответ на вопрос не надо, нужно чтобы ты выделил чанки\n"
        "Вопрос: {query_str}\n"
        "Твое рассуждение: "
)


# #не ворк
# RAG_PROMPT_1_shot = (
#     "Контекстная информация снизу. (набор документов чанков)\n"
#     "---------------------\n"
#     "{context_str}\n"
#     "---------------------\n"
#     "Используя только данную информацию контекст, "
#     "Ответь на вопрос. Дай прямой ответ. не генерируй markdown. Если этих документов недостаточно, чтобы ответить на вопрос, отказывайся отвечать.\n"
#     "Пример: вопрос:{q_shot}; эталон ответ:{tgt_shot}"
#     "конец примера.\n"
#     "Вопрос: {query_str}\n"
#     "Ответ: "
# )


In [23]:
import copy
def end2end_pipe(user_question, prompt_reasoning=REASONING_PROMPT, prompt=RAG_PROMPT):
    texts_orig, inds_orig = call_pipe(indexes, texts, user_question, k1=50, k2=10)
    texts_rer = copy.deepcopy(texts_orig)
    for j in range(len(texts_rer)):
        texts_rer[j] = f'Чанк {j+1}: {texts_rer[j]}'
    print(prompt_reasoning)
    prompt_desired = prompt_reasoning.format(
        context_str = '\n'.join(texts_rer),
        query_str = user_question
    )
    output_chunks_related = generate(llm, tokenizer, generation_config, prompt_desired)
    prompt_desired_rag = prompt.format(
        context_str = '\n'.join(texts_rer),
        query_str = user_question
    )
    output_rag = generate(llm, tokenizer, generation_config, prompt_desired_rag)
    return output_chunks_related, output_rag, inds_orig

In [87]:
import re
def parse_chunks_true(string, relev_inds):
    splitted = [text.lower() for text in string.split('.')[::-1]]
    try:
        for text in splitted:
            relevs = list(map(int, re.findall(r'\d+', text)))
            print(relevs)
            if len(relevs) > 0:
                print([relev_inds[relev_by_gpt - 1] for relev_by_gpt in relevs])
                return list(set([relev_inds[relev_by_gpt - 1] for relev_by_gpt in relevs]))
    except(Exception) as ex:
        print(ex)
        return []
    return []
            

In [1]:
from sklearn.metrics import fbeta_score

def calc_precision(true_, pred_):
    inter = set(true_) & set(pred_)
    return len(inter) / len(pred_) 

def calc_recall(true_, pred_):
    inter = set(true_) & set(pred_)
    return len(inter) / len(true_)

def calc_f_beta(PR, RE, beta=2, eps=0.01):
    ch = (1 + beta**2) * (PR * RE)
    znam = (beta ** 2) * PR + RE + eps
    return ch / znam

In [73]:
def take_threshold(
        retrieval_indexes,
        scores,
        corpus,
        urls,
        thres=0.5
    ):
    retrieval_indexes = np.array(retrieval_indexes)
    scores = np.array(scores)
    thresholded = retrieval_indexes[scores > thres]
    return thresholded
    

In [57]:
parse_chunks_true('''Для ответа на вопрос о том, как получить КЭП от Тинькофф, если пользователь не находится в Москве, необходимо рассмотреть чанки, содержащие информацию о процессе получения КЭП вне Москвы.

1. Чанк 1: Упоминается, что если пользователь не в Москве, он может подать заявку через личный кабинет Тинькофф Бизнеса и получить КЭП в удостоверяющем центре «Основание» в своем регионе.
2. Чанк 2: Описывает процесс подачи заявки на КЭП через личный кабинет Тинькофф Бизнеса и получения подписи в филиале «Основание» в другом городе.
3. Чанк 3: Подробно описывает процедуру получения КЭП в филиале «Основание» в своем городе, если пользователь не в Москве.
4. Чанк 7: Также содержит подробную инструкцию о том, как получить КЭП в филиале «Основание» в другом городе.

Чанки 2, 3 и 7 наиболее релевантны, поскольку они предоставляют конкретные шаги для получения КЭП в случае, когда пользователь не находится в Москве. Чанк 1 предоставляет общую информацию, но менее детализирован.

Релевантные чанки по убыванию релевантности: 3, 7, 2.''', [2, 2, 2, 4, 5, 6, 7])

[]
[3, 7, 2]


[2, 7]

In [ ]:
FINAL

In [98]:
import tqdm
from bert_score import score
import pandas as pd
import time
import copy

df = pd.read_csv("bench_final.csv")

logs = []
final_df = []
def benchmark(k = 12):
    bert_scores = []
    timings = []
    hits = 0
    gpt_metric = 0
    
    for i, row in tqdm.tqdm(df.iterrows()):
        user_question = row["Вопрос"]
        
        start_time = time.time()

        output_chunks_related, output_rag, inds_orig = end2end_pipe(user_question)
        parsed_urls_ids = parse_chunks_true(output_chunks_related, inds_orig)
        urls_relev = [urls[url_id] for url_id in parsed_urls_ids]
        final_df.append([user_question, output_rag, urls_relev])
        elapsed_time = time.time() - start_time
        timings.append(elapsed_time)
        if i % 10 == 0:
            print(i)

        logs.append(
            {
                'i': i,
                'output': output_rag,
                'model_name': model_name,
                'rag_prompt': RAG_PROMPT,
                'reasoning': output_chunks_related,
                'chunks': list(inds_orig),
                'k': 10,
                'chunk_overlap':chunk_overlap,
                'chunk_size':chunk_size
            }
        )
    #no need
    # avg_bert_score = sum(bert_scores) / len(bert_scores)
    avg_time = sum(timings) / len(timings)
    return _, _, avg_time, logs

In [90]:
generation_config.temperature=0.2
results = benchmark(1)

In [125]:
final_df.to_csv('results.csv', index=False)

In [ ]:
for i in range(len(kkk)):
    parsed_urls_ids = parse_chunks_true(kkk[i], logs[i]['chunks'])
    urls_relev = [urls[url_id] for url_id in parsed_urls_ids]
    final_df.loc[i, 'Urls'] = urls_relev

In [114]:
zhiest = pd.DataFrame(yyoy, columns=[
    'Question',
    'Predict',
    'Urls'
])

In [115]:
zhiest.to_csv('finalOur.csv', index=False)

In [ ]:
import json

def save_as_json(filename, data):
    try:
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4) 
        print(f"Data successfully written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")


In [ ]:
for i in range(len(logs)):
    logs[i]['chunks'] = list(map(int, logs[i]['chunks']))

In [ ]:
save_as_json('ideal_prompt_100_t_pro_4bit.json', logs)

In [ ]:
#contextual creater
def context_creater(chunk, full_chunk):
    prompt = 'Тебе даны 2 текста. первый текст взят из второго. Твоя задача дать очень маленький контекст для первого текста на основе второго' \
                'чтобы первый текст был осмысленным. НЕ УПОМЯНАЙ ПЕРВЫЙ ТЕКСТ И НЕ УПОМЯНАЙ ВТОРОЙ ТЕКСТ"\n' \
                'Первый текст: {chunk}\n\n' \
                'Второй текст: {full_chunk}\n\nКонтекст:'
    res = generate(model, 
                   tokenizer, 
                   generation_config, 
                   prompt.format(chunk=chunk, full_chunk=full_chunk))
    return res    


In [ ]:
from collections import Counter

In [ ]:
dd = Counter(indexes)
curr = 0
chunks_d = []
for ind, (k, v) in enumerate(dd):
    if v != 1:
        for j in range(v):
            res = context_creater(texts[curr + j], corpus[indexes[curr + j]])
            chunks_d.append((curr + j, res))
    curr += v
save_as_json('logical_chunks', chunks_d)

In [ ]:
#конкатенация контекста.
def concat_context(contexts, texts, delim='.'):
    for id_chunk, context in contexts:
        texts[id_chunk] = context + delim + texts[id_chunk]